In [1]:
import numpy as np
import gzip
import pandas as pd
from collections import defaultdict
from numpy import median

dataDir = "/Users/shiqi/Downloads/cse258/assignment1/"
path = dataDir + "trainInteractions.csv.gz"
f = gzip.open(path, 'rt', encoding="utf8")

header = f.readline()
header = header.strip().split('\t')
header
# ['user_id,recipe_id,date,rating']
dataset = []

for line in f:
    fields = line.strip().split('\t')
    d = dict(zip(header, fields))
    dataset.append(d)
    
# Split the data into training and validation sets
data_train = dataset[:400000]
data_valid =dataset[400000:]
    

In [2]:
# Q1 uncooked (user,item) pair in validation
valid = []

for d in data_valid:
    field = d['user_id,recipe_id,date,rating']
    valid.append(field)
    
user_id = []
item_id = []
date = []
rating = []
for d in valid:
    sl = d.split(",")
    u = sl[0]
    i = sl[1]
    da = sl[2]
    ra = sl[3]
    user_id.append(u)
    item_id.append(i)
    date.append(da)
    rating.append(ra)
    
unique_users = list(set(user_id))
unique_items = list(set(item_id))
    
Pair_0 = []
item_id_array = np.array(item_id)
user_id_array = np.array(user_id)
unique_items_array = np.array(unique_items)

In [3]:
# sample a negative entry by randomly choosing a recipe that user hasn’t cooked
from random import sample
for d in range(len(unique_users)):
    #ct = ct+1
    #print(ct)
    item_1 = item_id_array[user_id_array == unique_users[d]]
    #item_0 = [it for it in unique_items if it not in item_1.tolist()]  list difference  
    item_0 = np.setdiff1d(unique_items_array, item_1).tolist()
    pair_0 = [(unique_users[d],l) for l in sample(item_0,len(item_1))]
    Pair_0 += pair_0
   
user_id_0 = []
item_id_0 = []
for (u,i) in Pair_0:
    user_id_0.append(u)
    item_id_0.append(i)


In [4]:
cook_0 = [0] * len(user_id_0)
cook_1 = [1] * len(user_id)

In [5]:
# save the validation with 0 cooks to pandas dataframe
valid_user_id_w0 = user_id + user_id_0
valid_item_id_w0 = item_id + item_id_0
valid_cook_w0 = cook_1 + cook_0

dictionary = {'user_id': valid_user_id_w0,'recipe_id':valid_item_id_w0,'cook':valid_cook_w0}  
valid_w0 = pd.DataFrame(dictionary) 

In [6]:
# Q1 Evaluate the performance (accuracy) of the baseline model on the trainning set
train = []

for d in data_train:
    field = d['user_id,recipe_id,date,rating']
    train.append(field)
    
user_id = []
item_id = []
date = []
rating = []
for d in train:
    sl = d.split(",")
    u = sl[0]
    i = sl[1]
    da = sl[2]
    ra = sl[3]
    user_id.append(u)
    item_id.append(i)
    date.append(da)
    rating.append(ra)
    
cook_train = [1] * len(user_id)


In [7]:
# save the train data to pandas dataframe
dictionary = {'user_id': user_id,'recipe_id':item_id,'cook':cook_train}  
train = pd.DataFrame(dictionary) 

In [9]:
# Would-cook baseline: 
# just rank which recipes are popular and which are not, 
# and return '1' if a recipe is among the top-ranked

Count = train.groupby('recipe_id')[['cook']].sum().reset_index()
Count = Count.sort_values('cook', ascending=[False])
totalCooked = train['cook'].sum()
mostPopular = Count.values

return1 = set()
count = 0
for ic, i in mostPopular:
  count += i
  return1.add(ic)
  if count > totalCooked/2: break

pred_1 = valid_w0['recipe_id'].isin(return1)
valid_w0['prediction'] = 0
valid_w0.loc[pred_1,'prediction']= 1
ac = valid_w0['cook'] == valid_w0['prediction']


Q1. Although we have built a validation set, it only consists of positive samples. For this task we also need examples of user/item pairs corresponding to recipes that weren’t cooked. For each entry (user,recipe) in the validation set, sample a negative entry by randomly choosing a recipe that user hasn’t cooked.1 Evaluate the performance (accuracy) of the baseline model on the validation set you have built (1 mark).

In [10]:
ac.sum()/len(ac)
# accuracy 0.62352

0.61477

Q2. Assuming that the ‘non-made’ test examples are a random sample of user-recipe pairs, this threshold may not be the best one. See if you can find a better threshold and report its performance on your validation set (1 mark).

In [11]:
return1 = set()
count = 0
for ic, i in mostPopular:
  count += i
  return1.add(ic)
  if count > totalCooked/(3/5): break

pred_1 = valid_w0['recipe_id'].isin(return1)
valid_w0['prediction'] = 0
valid_w0.loc[pred_1,'prediction']= 1
ac = valid_w0['cook'] == valid_w0['prediction']

In [12]:
# accuracy 0.55215
print('use 3/5 of the totalCooked, accuracy is:', ac.sum()/len(ac))

use 3/5 of the totalCooked, accuracy is: 0.551135


Q3. An alternate baseline than the one provided might make use of the Jaccard similarity (or another sim- ilarity metric). Given a pair (u,g) in the validation set, consider all training items g′ that user u has cooked. For each, compute the Jaccard similarity between g and g′, i.e., users (in the training set) who have made g and users who have made g′. Predict as ‘made’ if the maximum of these Jaccard similarities exceeds a threshold (you may choose the threshold that works best). Report the performance on your validation set.

In [13]:
# Jaccard Similarity
# training set
usersPerItem_train = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser_train = defaultdict(set) # Maps a user to the items that they rated
cookDict_train = {} # To retrieve a rating for a specific user/item pair
user_train = train['user_id'].values.tolist()
item_train = train['recipe_id'].values.tolist()
cook_train = train['cook'].values.tolist()

for i in range(len(user_train)):
    user_t = user_train[i]
    item_t = item_train[i]
    usersPerItem_train[item_t].add(user_t)
    itemsPerUser_train[user_t].add(item_t)
    cookDict_train[(user_t,item_t)] = cook_train[i]

In [14]:
# validation set
usersPerItem_valid = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser_valid = defaultdict(set) # Maps a user to the items that they rated
cookDict_valid = {} # To retrieve a rating for a specific user/item pair
user_valid = valid_w0['user_id'].values.tolist()
item_valid = valid_w0['recipe_id'].values.tolist()
cook_valid = valid_w0['cook'].values.tolist()

for i in range(len(user_valid)):
    user_v = user_valid[i]
    item_v = item_valid[i]
    usersPerItem_valid[item_v].add(user_v)
    itemsPerUser_valid[user_v].add(item_v)
    cookDict_valid[(user_v,item_v)] = cook_valid[i]

In [15]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

sim_train = []
for d,g in zip(user_train,item_train):
    g_prime = itemsPerUser_train[d] - {g}
    if len(g_prime) == 0:
        sim_train.append(0)
    else:
        u = usersPerItem_train[g] - {d}
        similarities = []
        for g_ in list(g_prime):
            u_ = usersPerItem_train[g_] - {d}      
            similarities.append(Jaccard(u, u_))
        sim = max(similarities)
        sim_train.append(sim)

In [17]:
# let the median of similarities in the training be the shreshold
Lambda = median(sim_train)

In [18]:
# predictions using Jaccard in the validation

sim_valid = [0] * len(user_valid)
pred_valid = [0] * len(user_valid)
ct = 0
for d,g in zip(user_valid,item_valid):
    if (d in set(user_train)) & (g in set(item_train)):
        g_prime = itemsPerUser_train[d]
        u = usersPerItem_train[g]
        similarities = []
        for g_ in list(g_prime):
            u_ = usersPerItem_train[g_]        
            similarities.append(Jaccard(u, u_))
        sim = max(similarities)
        sim_valid[ct] = sim
        if sim > Lambda:
            pred_valid[ct] = 1
    ct = ct + 1

In [19]:
# the accuracy of Jaccrad Similarity Predictions on validation
ac = sum((np.array(pred_valid) - np.array(cook_valid)) == 0) / len(pred_valid)
print('use Jaccard similarity, accuracy is:', ac)

use Jaccard similarity, accuracy is: 0.570815


Q4.Improve the above predictor by incorporating both a Jaccard-based threshold and a popularity based threshold. Report the performance on your validation set (1 mark).

In [ ]:
mostPopular = pd.DataFrame(mostPopular,columns = ['item_id','count'])
mostPopular['cum_sum'] = mostPopular['count'].cumsum()

Percentile_in_Mostpopular =[]

for i in item_valid:
    percentile = 1 - mostPopular.loc[mostPopular['item_id'] == i,'cum_sum']/400000
    if len(percentile) ==0:
        Percentile_in_Mostpopular.append(0)
    else:
        Percentile_in_Mostpopular.append(percentile.values[0])

In [34]:
a = np.array(sim_valid).reshape(-1,1)
b = np.array(Percentile_in_Mostpopular).reshape(-1,1)
X = np.hstack((a,b))
y = np.array(cook_valid).reshape(-1,1)

In [36]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X, y)
pred_4 = clf.predict(X)

/Users/shiqi/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/shiqi/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [38]:
ac = sum((np.array(pred_4) - np.array(cook_valid)) == 0) / len(cook_valid)
print('Jaccard_sim and Mostpopular as logistic Feature, accuracy is:', ac)

Jaccard_sim and Mostpopular as logistic Feature, accuracy is: 0.613595


Q5.To run our model on the test set, we’ll have to use the files ‘stub Made.txt’ to find the user id/recipe id pairs about which we have to make predictions. Using that data, run the above model and upload your solution to Kaggle. Tell us your Kaggle user name (1 mark). 

In [39]:
# load data from test
user_test = []
item_test = []

for l in open("stub_Made.txt"):
  if l.startswith("user_id"):
    #header
    continue
  u,i = l.strip().split('-')
  user_test.append(u)
  item_test.append(i)

In [40]:
# compute test similarity
sim_test = [0] * len(user_test)

ct = 0
for d,g in zip(user_test,item_test):
    if (d in set(user_train)) & (g in set(item_train)):
        g_prime = itemsPerUser_train[d]
        u = usersPerItem_train[g]
        similarities = []
        for g_ in list(g_prime):
            u_ = usersPerItem_train[g_]        
            similarities.append(Jaccard(u, u_))
        sim = max(similarities)
        sim_test[ct] = sim
    ct = ct + 1

In [41]:
# compute percentile of test
Percentile_in_Mostpopular_test =[]

for i in item_test:
    percentile = 1 - mostPopular.loc[mostPopular['item_id'] == i,'cum_sum']/400000
    if len(percentile) ==0:
        Percentile_in_Mostpopular_test.append(0)
    else:
        Percentile_in_Mostpopular_test.append(percentile.values[0])

In [43]:
a = np.array(sim_test).reshape(-1,1)
b = np.array(Percentile_in_Mostpopular_test).reshape(-1,1)
X_test = np.hstack((a,b))

In [44]:
pred_5 = clf.predict(X_test)

dictionary = {'user_id': user_test,'recipe_id':item_test,'prediction':pred_5}  
dataframe = pd.DataFrame(dictionary)
dataframe['user_id-recipe_id'] = dataframe[['user_id', 'recipe_id']].apply(lambda x: '-'.join(x), axis=1)
df = dataframe[['user_id-recipe_id','prediction']]
df.to_csv('test_pred.csv',index = False) 

Q5. Kaggle user id: ShiqiB, accuracy in test is 0.7088

Q9 Fit a predictor of the form rating(user,item) ~ alpha + B_u + B_i, by fitting the mean and the two bias terms as described in the lecture notes. Use a regularization
parameter of λ = 1. Report the MSE on the validation set (1 mark).

In [45]:
# Q9 
# use the trainning data

train = []

for d in data_train:
    field = d['user_id,recipe_id,date,rating']
    train.append(field)
    
user_id = []
item_id = []
date = []
rating = []
for d in train:
    sl = d.split(",")
    u = sl[0]
    i = sl[1]
    da = sl[2]
    ra = sl[3]
    user_id.append(u)
    item_id.append(i)
    date.append(da)
    rating.append(ra)

In [46]:
# Simple (bias only) latent factor-based recommender
import scipy
import scipy.optimize    

itemsPerUser = defaultdict(list)
usersPerItem = defaultdict(list)

for u, i in zip(user_id,item_id):
    itemsPerUser[u].append(i)
    usersPerItem[i].append(u)
    
labels = [int(d) for d in rating]
ratingMean = sum(labels) / len(labels)

N = len(train)
nUsers = len(itemsPerUser)
nItems = len(usersPerItem)
users = list(itemsPerUser.keys())
items = list(usersPerItem.keys())
    
alpha = ratingMean

userBiases = defaultdict(float)
itemBiases = defaultdict(float)

In [47]:
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item]
    
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))
    
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(u, i) for u,i in zip(user_id,item_id)]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost
 
    
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(dataset)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for u,i,r in zip(user_id,item_id,labels):
        pred = prediction(u, i)
        diff = pred - r
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
    for d in userBiases:
        dUserBiases[d] += 2*lamb*userBiases[d]
    for p in itemBiases:
        dItemBiases[p] += 2*lamb*itemBiases[p]
    dtheta = [dalpha] + [dUserBiases[d] for d in users] + [dItemBiases[p] for p in items]
    return np.array(dtheta)

In [48]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems),
                             derivative, args = (labels, 1))

MSE = 0.8987313599958769
MSE = 0.8856358581692653
MSE = 0.8986218156135961
MSE = 0.8986223585785804
MSE = 0.8986218283827365
MSE = 0.8986218159274242
MSE = 0.8986218156213774
MSE = 0.8986218156138821
MSE = 0.8986218156135792
MSE = 0.8986218156135792
MSE = 0.8986218156137126
MSE = 0.8986218156135766
MSE = 0.8986218156135765
MSE = 0.8986218156137227
MSE = 0.898621815613584
MSE = 0.8986218156135765


(array([ 4.58080000e+00, -6.90340584e-05, -6.47033902e-06, ...,
        -1.16446855e-06,  8.40470414e-07, -1.16446855e-06]),
 0.8986658023286048,
 {'grad': array([ 1.58518187e-04, -4.20211595e-07, -3.23406545e-08, ...,
         -5.80250330e-09,  7.32538905e-09, -6.02742858e-09]),
  'task': b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH',
  'funcalls': 16,
  'nit': 2,
  'warnflag': 0})

In [49]:
# MSE on the validation set
valid = []

for d in data_valid:
    field = d['user_id,recipe_id,date,rating']
    valid.append(field)
    
user_id_valid = []
item_id_valid = []
date_valid = []
rating_valid = []
for d in valid:
    sl = d.split(",")
    u = sl[0]
    i = sl[1]
    da = sl[2]
    ra = sl[3]
    user_id_valid.append(u)
    item_id_valid.append(i)
    date_valid.append(da)
    rating_valid.append(ra)
    
valid_pred = []
for u,i in zip(user_id_valid,item_id_valid):
    if (u in user_id) & (i in item_id):
        pred = prediction(u,i)
    else:
        pred = 0
    valid_pred.append(pred)


valid_labels = [int(d) for d in rating_valid]
mse = MSE(valid_pred, valid_labels)

In [50]:
print('MSE on the validation when lambda = 1:', mse)

MSE on the validation when lambda = 1: 4.424182115391335


Q10.Report the user and recipe IDs that have the largest and smallest values of β (1 mark)

In [51]:
min_i = min(itemBiases, key=itemBiases.get)
min_u = min(userBiases, key=userBiases.get)
print('user_id with smallest value:', min_u)
print('recipe_id with smallest value:', min_i)

user_id with smallest value: 70705426
recipe_id with smallest value: 29147042


In [52]:
max_i = max(itemBiases, key=itemBiases.get)
max_u = max(userBiases, key=userBiases.get)
print('user_id with largest value:', max_u)
print('recipe_id with largest value:', max_i)

user_id with largest value: 32445558
recipe_id with largest value: 98124873


Q11.Find a better value of λ using your validation set. Report the value you chose, its MSE, and upload your solution to Kaggle by running it on the test data (1 mark).

In [53]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems),
                             derivative, args = (labels,0.00001)) 

MSE = 0.8987313599956165
MSE = 0.8856358581703624
MSE = 1.0746606639793392
MSE = 0.8832581770796194
MSE = 0.8771058156117816
MSE = 0.873528718514732
MSE = 0.862320034372388
MSE = 0.8512624763515296
MSE = 0.8444868753600093
MSE = 0.8228016646938193
MSE = 0.8067877503919217
MSE = 0.7856040265163795
MSE = 0.7726711764546196
MSE = 0.7645020637406588
MSE = 0.7526559733162275
MSE = 2.332933414419877
MSE = 0.7525687420785727
MSE = 0.7485172844293243
MSE = 0.745751976602477
MSE = 0.7388906345384213
MSE = 0.7337763733758575
MSE = 0.7286398545569337
MSE = 0.7184982274027016
MSE = 0.7065799398162318
MSE = 0.6997977568019986
MSE = 0.6902915107464591
MSE = 0.6867562582280317
MSE = 0.7256021731133394
MSE = 0.6866777931626011
MSE = 0.6851839999427446
MSE = 0.6799994596446733
MSE = 0.6779274036896351
MSE = 0.676086853630142
MSE = 0.674883892415983
MSE = 0.6770513151037971
MSE = 0.6720589736972918
MSE = 0.6687911089972433
MSE = 0.6666579688920391
MSE = 0.6666345947454777
MSE = 0.6663721228203081
MSE = 

(array([ 4.44422718,  0.0110169 , -0.17181633, ..., -0.05755092,
         0.02968592,  0.00505276]),
 0.6920247959690112,
 {'grad': array([-4.39403112e-04,  1.82387726e-05, -2.42875820e-06, ...,
          6.63247725e-07, -6.40542839e-07,  1.79803282e-07]),
  'task': b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH',
  'funcalls': 94,
  'nit': 64,
  'warnflag': 0})

In [54]:
valid_pred = []
for u,i in zip(user_id_valid,item_id_valid):
    if (u in user_id) & (i in item_id):
        pred = prediction(u,i)
    else:
        pred = 0
    valid_pred.append(pred)


valid_labels = [int(d) for d in rating_valid]
mse = MSE(valid_pred, valid_labels)

In [55]:
print('MSE on the validation when lambda = 0.00001:', mse)
# lambda = 0.00001: mse = 4.3638
# lambda = 0.001: mse = 4.3995
# lambda = 100: mse = 4.4242

MSE on the validation when lambda = 0.00001: 4.3677161976797985


In [56]:
# predictions in test data
user_test = []
item_test = []

for l in open("stub_Rated.txt"):
  if l.startswith("user_id"):
    #header
    continue
  u,i = l.strip().split('-')
  user_test.append(u)
  item_test.append(i)

In [57]:
test_pred = []
for u,i in zip(user_test,item_test):
    if (u in user_id) & (i in item_id): # in the training data
        pred = prediction(u,i)
    else:
        pred = 0
    test_pred.append(pred)

In [58]:
dictionary = {'user_id': user_test,'recipe_id':item_test,'prediction':test_pred}  
dataframe = pd.DataFrame(dictionary)
dataframe['user_id-recipe_id'] = dataframe[['user_id', 'recipe_id']].apply(lambda x: '-'.join(x), axis=1)
df = dataframe[['user_id-recipe_id','prediction']]
df.to_csv('test_pred.csv',index = False) 